In [2]:
from diffusers import AutoencoderKL
import torch
from PIL import Image
import numpy as np
from torch.cuda.amp import autocast
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
torch_device = "cuda"
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to(torch_device)

In [4]:
@torch.no_grad()
def to_latents(img:Image):
  np_img = (np.array(img).astype(np.float32) / 255.0) * 2.0 - 1.0
  np_img = np_img[None].transpose(0, 3, 1, 2)
  torch_img = torch.from_numpy(np_img)
  with autocast():
    generator = torch.Generator("cuda").manual_seed(0)
    latents = vae.encode(torch_img.to(vae.dtype).to(torch_device)).latent_dist.sample(generator=generator)
  return latents
@torch.no_grad()
def to_img(latents):
  with autocast():
    torch_img = vae.decode(latents.to(vae.dtype).to(torch_device)).sample
  torch_img = (torch_img / 2 + 0.5).clamp(0, 1)
  np_img = torch_img.cpu().permute(0, 2, 3, 1).detach().numpy()[0]
  np_img = (np_img * 255.0).astype(np.uint8)
  img = Image.fromarray(np_img)
  return img

In [62]:
with tf.device("CPU:0"):
    (ds_train, ds_test), ds_info = tfds.load(
        "imagenette", split=['train','validation'], with_info=True, as_supervised=True
    )
    N1 = len(list(ds_train))
    N2 = len(list(ds_test))
    z1 = torch.zeros((N1,4,64,64))
    y1 = np.zeros((N1),dtype=int)
    z2 = torch.zeros((N2,4,64,64))
    y2 = np.zeros((N2),dtype=int)
    for i_img, (x,y) in enumerate(ds_train.take(-1)):
        resized = tf.image.resize(x, (512,512))
        z1[i_img,:,:,:] = to_latents(resized)
        y1[i_img] = y

    for i_img, (x,y) in enumerate(ds_test.take(-1)):
        resized = tf.image.resize(x, (512,512))
        z2[i_img,:,:,:] = to_latents(resized)
        y2[i_img] = y
        pass
ds2_train = tf.data.Dataset.from_tensor_slices((z1, y1))
ds2_test = tf.data.Dataset.from_tensor_slices((z2, y2))
ds2_train.save('latent_train')
ds2_test.save('latent_test')

2023-03-05 21:21:22.001884: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 620560384 exceeds 10% of free system memory.
2023-03-05 21:21:22.379476: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 620560384 exceeds 10% of free system memory.
